In [1]:
# Advent of Code, Transparent Origami Day 13 - Jim Carson. 
# Rewrote to use numpy arrays (Thanks, Janet for showing me that!),
# debugging output colorized from day 11 code
#
# This is excessively long
# 

import numpy as np

DEBUG = False

# This block is used to pretty-print the matrices we're debugging.  
# See ANSI codes https://jupyterbook.org/content/code-outputs.html
fgcolor = "1;31" # Red foreground text
formatstring = "\x1b[1;31m{0:>1}\x1b[m"

class sheet:
    def __init__(self, filename):
        # Read file
        with open(filename,"r") as fp:
            f = fp.read()
        # Split input file by blank line.
        coords, instr = f.split("\n\n")
        # Create data structures to read
        self.coordinates = []
        self.instructions = []
        self.first_instructions = []
        # Add coordinates to list
        for i in coords.splitlines():
            x,y = map(int, i.split(","))
            self.coordinates.append((x,y))      
        self.max_x = max([x for x,y in self.coordinates])
        self.max_y = max([y for x,y in self.coordinates])
        if DEBUG:
            print(self.max_x, self.max_y)
        # Create the paper
        self.paper = np.zeros((self.max_x + 1, self.max_y + 1), dtype=int)
        # Color in the "dots"
        for i in self.coordinates:
            self.paper[i] = 1
        # Add the list of folding instructions
        for i in instr.splitlines():
            axis, position = i.replace("fold along ","").split("=")
            if DEBUG:
                print(axis, position)
            self.instructions.append((axis,int(position)))
            if not self.first_instructions:
                self.first_instructions.append((axis, position))
    
    # Since we have a numpy array, we can slice it, flip it, and do a logical or
    def fold(self, axis, position, DEBUG=DEBUG):
        if axis == 'y':
            top = self.paper[:, 0:position]
            bottom =  np.flip(self.paper[:, position + 1 :], axis=1)
            self.paper = top | bottom
        else:
            left = self.paper[0:position, :]
            right = np.flip(self.paper[position + 1 :, :], axis=0)
            self.paper = left | right
        if DEBUG:
            self.color_grid(match=1)

    # output a colorized grid.
    def color_grid(self, **kwargs):
        x_value_highlight = kwargs.pop('x', None)
        y_value_highlight = kwargs.pop('y', None)
        match_arg = kwargs.pop("match", None)
        printas = kwargs.pop("printas", "01")
        
        for j in range(self.paper.shape[1]):
            line = ""
            for i in range(self.paper.shape[0]):         
                if i == x_value_highlight or j == y_value_highlight or match_arg == self.paper[i,j]:
                    line = line + (formatstring.format(str(printas[self.paper[i,j]])))
                else:
                    line = line + ("{0:>1}".format(str(printas[self.paper[i,j]])))
            print(line)
        
    def print(self):
        dot = ".*"
        print("\n".join("".join([dot[i] for i in line]) for line in self.paper.T))


In [2]:
# Test example, showing colorization
s = sheet("input_files/day13.test.txt")
(axis, position) = s.instructions[0]
s.color_grid(match=1, y=position)

00010010010
00001000000
00000000000
10000000000
00010000101
00000000000
00000000000
00000000000
00000000000
00000000000
01000010110
00001000000
00000010001
10000000000
10100000000


In [3]:
# fold once
s.fold(axis, position)
s.color_grid(match=1)

10110010010
10001000000
00000010001
10001000000
01010010111
00000000000
00000000000


In [4]:
# Part 1: 678
s = sheet("input_files/day13.txt")
s.fold(*s.instructions[0])
np.sum(s.paper>0)

678

In [5]:
# Part 2: ECFHLHZF
s = sheet("input_files/day13.txt")
for axis, position in s.instructions:
    s.fold(axis, position)
s.color_grid(match=1, printas=" #")

####  ##  #### #  # #    #  # #### #### 
#    #  # #    #  # #    #  #    # #    
###  #    ###  #### #    ####   #  ###  
#    #    #    #  # #    #  #  #   #    
#    #  # #    #  # #    #  # #    #    
####  ##  #    #  # #### #  # #### #    
